In [4]:
# !pip install gspread oauth2client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.5/195.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 17.6 MB/s eta 0:00:00
Using cached rsa-4.9-py3-none-any.whl (34 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 17.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install gspread oauth2client pandas


  Obtaining dependency information for gspread from https://files.pythonhosted.org/packages/5a/46/f9fe3b4834ab93a0e621c942fbc289fc2fc9626c77c2d36e41937be454a8/gspread-6.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for oauth2client from https://files.pythonhosted.org/packages/95/a9/4f25a14d23f0786b64875b91784607c2277eff25d48f915e39ff0cff505a/oauth2client-4.1.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth>=1.12.0 from https://files.pythonhosted.org/packages/e7/00/85c22f7f73fa2e88dfbf0e1f63c565386ba40e0264b59c8a4362ae27c9fc/google_auth-2.32.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib>=0.4.1 from https://files.pythonhosted.org/packages/1a/8e/22a28dfbd218033e4eeaf3a0533b2b54852b6530da0c0fe934f0cc494b29/google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for httplib2>=0.9.1 from https://files.pythonhosted.org/packages/a8/6c/d2fbdaaa5959339d53ba38e94c

In [35]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import json

def upload_google (function, usage):
    clean_path0 = f"response_data/split_conversations/{function}/{usage}.json"
    with open(clean_path0, "r") as jfile:
        conver_data=json.load(jfile)

    data = []
    for key, conversations in conver_data.items():
        row_trial = []
        for conversation in conversations:
            cleaned_conversation = conversation.replace("\n\n", " ").replace("\n", " ")
            # Create a list with the structure [Category, Conversation, '', '']

            if key not in row_trial:
                row = [key, cleaned_conversation, '', '']
                row_trial.append(key)
            else:
                row = ['', cleaned_conversation, '', '']
            data.append(row)
            data.append(['', '', '', ''])
            data.append(['', '', '', ''])

    # Convert data to DataFrame
    df = pd.DataFrame(data, columns=['Category', 'Conversation', 'FQ on function level', 'FQ on feedback'])

    # # Display the DataFrame
    # print(df)

    # Google Sheets setup
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(' ', scope)#change key
    client = gspread.authorize(credentials)

    # Create a new Google Sheet
    spreadsheet = client.create(f'{function}_{usage}_scheme')

    # Share the sheet with email address
    spreadsheet.share('', perm_type='user', role='writer')

    # Select the first sheet
    sheet = spreadsheet.get_worksheet(0)

    # Update the sheet with data
    sheet.update([df.columns.values.tolist()] + df.values.tolist())

    print(f'Data {function}_{usage} uploaded to Google Sheets.')


In [36]:
functions = ["communication","mobility", "self-care"]
us_path = ["test","val"]
# Flatten the conversation data to list of lists

for function in functions:
    for usage in us_path:
        upload_google (function, usage)
        
# function2 = "communication"
# usage2 = "val"
# upload_google (function2, usage2)

Data communication_test uploaded to Google Sheets.
Data communication_val uploaded to Google Sheets.
Data mobility_test uploaded to Google Sheets.
Data mobility_val uploaded to Google Sheets.
Data self-care_test uploaded to Google Sheets.
Data self-care_val uploaded to Google Sheets.
